In [1]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

timestr = time.strftime("%Y%m%d-%H%M%S")
df = pd.read_csv('data/example_pcap3.csv')

#dataset = pd.get_dummies(dataset)
#dataset = dataset.replace(np.nan, 0)
#dataset = dataset.to_numpy()

df_quic = df[df["quic"] == "quic"] #only need quic packets bc these are the only packets for the video

df_quic = df_quic.reset_index(drop=True) #reset df indices after dropping rows

client = df_quic['ip.src'][0] #client sends first request to server

df = df[df['ip.dst'] == client] #only look at packets incoming to the client

df = df.reset_index(drop=True) #reset df indices after dropping rows


#go through and calculate chunk sizes

chunk_size_data = []

last = 0 #0=udp last, 1=quic last
chunk_size_udp = 0
chunk_size_quic = 0
prev_start_time_udp = 0
prev_start_time_quic = 0
for i in range(len(df)):
    if(not("eth:ethertype:ip:udp:data" == df['frame.protocols'][i] or "eth:ethertype:ip:udp:quic" == df['frame.protocols'][i])):
        continue
    else:
        if(prev_start_time_udp == 0 and "eth:ethertype:ip:udp:data" == df['frame.protocols'][i]):
            prev_start_time_udp = df['frame.time_relative'][i]
        if(prev_start_time_quic == 0 and "eth:ethertype:ip:udp:quic" == df['frame.protocols'][i]):
            prev_start_time_quic = df['frame.time_relative'][i]
        if("eth:ethertype:ip:udp:data" == df['frame.protocols'][i] and df['frame.time_relative'][i]>prev_start_time_udp + 0.1): #more than 0.3s since last packet, means new chunk
            chunk_size_data.append(chunk_size_udp)
            chunk_size_udp = 0
            prev_start_time_udp = df['frame.time_relative'][i]
            last = 0
        if("eth:ethertype:ip:udp:quic" == df['frame.protocols'][i] and df['frame.time_relative'][i]>prev_start_time_quic + 0.1): #more than 0.3s since last packet, means new chunk
            chunk_size_data.append(chunk_size_quic)
            chunk_size_quic = 0
            prev_start_time_quic = df['frame.time_relative'][i]
            last = 1
        if("eth:ethertype:ip:udp:data" == df['frame.protocols'][i]):
            chunk_size_udp += int(df['frame.len'][i])
            prev_start_time_udp = df['frame.time_relative'][i]
        if("eth:ethertype:ip:udp:quic" == df['frame.protocols'][i]):
            chunk_size_quic += int(df['frame.len'][i])
            prev_start_time_quic = df['frame.time_relative'][i]

if(last == 0):
    if(chunk_size_quic>0):
        chunk_size_data.append(chunk_size_quic)
    if(chunk_size_udp>0):
        chunk_size_data.append(chunk_size_udp)
else:
    if(chunk_size_udp>0):
        chunk_size_data.append(chunk_size_udp)
    if(chunk_size_quic>0):
        chunk_size_data.append(chunk_size_quic)

chunk_size_data=np.asarray(chunk_size_data)







#detect new chunk on special packet - not possible

'''
last = 0 #0 = udp, 1 = quic
chunk_size_udp = 0
chunk_size_quic = 0
prev_len_udp = 0
prev_len_quic = 0
#prev_start_time_udp = 0
#prev_start_time_quic = 0
for i in range(len(df)):
    if(not("eth:ethertype:ip:udp:data" == df['frame.protocols'][i] or "eth:ethertype:ip:udp:quic" == df['frame.protocols'][i])):
        continue
    else:
        if(df['ip.src'][i] == client): #new chunk
            if("eth:ethertype:ip:udp:data" == df['frame.protocols'][i] and prev_len_udp != df['frame.len'][i]): #new udp chunk
                if(chunk_size_udp == 0): #beginning of first chunk
                    continue
                chunk_size_data.append(chunk_size_udp)
                chunk_size_udp = 0
                prev_len_udp = df['frame.len'][i]
                #prev_start_time_udp = df['frame.time_relative'][i]
                last = 0
            if("eth:ethertype:ip:udp:quic" == df['frame.protocols'][i] and prev_len_quic != df['frame.len'][i]): #new quic chunk
                if(chunk_size_quic == 0): #beginning of first chunk
                    continue
                chunk_size_data.append(chunk_size_quic)
                chunk_size_quic = 0
                prev_len_quic = df['frame.len'][i]
                #prev_start_time_quic = df['frame.time_relative'][i]
                last = 1
        else: #same chunk
            #if(prev_start_time_udp == 0 and "eth:ethertype:ip:udp:data" == df['frame.protocols'][i]):
            #    prev_start_time_udp = df['frame.time_relative'][i]
            #if(prev_start_time_quic == 0 and "eth:ethertype:ip:udp:quic" == df['frame.protocols'][i]):
            #    prev_start_time_quic = df['frame.time_relative'][i]
            if("eth:ethertype:ip:udp:data" == df['frame.protocols'][i]):
                chunk_size_udp += int(df['frame.len'][i])
                #prev_start_time_udp = df['frame.time_relative'][i]
            else:
                chunk_size_quic += int(df['frame.len'][i])
                #prev_start_time_quic = df['frame.time_relative'][i]

if(last == 0):
    if(chunk_size_quic>0):
        chunk_size_data.append(chunk_size_quic)
    if(chunk_size_udp>0):
        chunk_size_data.append(chunk_size_udp)
else:
    if(chunk_size_udp>0):
        chunk_size_data.append(chunk_size_udp)
    if(chunk_size_quic>0):
        chunk_size_data.append(chunk_size_quic)'''



'''
chunk_size_data = []

chunk_size = 0
init = 1
#prev_start_time = 0
for i in range(len(df)):
    if(df['Info'][i] != "Protected Payload (KP0)"):
        if(df['Info'][i].startswith("Initial")): #new chunk
            if(init == 1):
                continue
            chunk_size_data.append(chunk_size)
            chunk_size = 0
                
        continue
        df = df[df['Destination'] == client]
    if(df['Info'][i] == "Protected Payload (KP0)" and df['Destination'][i] == client):
        if(init == 1):
            init = 0
        chunk_size += int(df['Length'][i])

chunk_size_data=np.asarray(chunk_size_data)
print(chunk_size_data)
print(chunk_size_data.shape)
'''

'\nchunk_size_data = []\n\nchunk_size = 0\ninit = 1\n#prev_start_time = 0\nfor i in range(len(df)):\n    if(df[\'Info\'][i] != "Protected Payload (KP0)"):\n        if(df[\'Info\'][i].startswith("Initial")): #new chunk\n            if(init == 1):\n                continue\n            chunk_size_data.append(chunk_size)\n            chunk_size = 0\n                \n        continue\n        df = df[df[\'Destination\'] == client]\n    if(df[\'Info\'][i] == "Protected Payload (KP0)" and df[\'Destination\'][i] == client):\n        if(init == 1):\n            init = 0\n        chunk_size += int(df[\'Length\'][i])\n\nchunk_size_data=np.asarray(chunk_size_data)\nprint(chunk_size_data)\nprint(chunk_size_data.shape)\n'

In [2]:
i=0
while(i<chunk_size_data.shape[0]):
    if chunk_size_data[i]<5000:
        chunk_size_data = np.delete(chunk_size_data, [i], 0)
    else:
        i+=1
print(chunk_size_data)
print(chunk_size_data.shape)

[ 154436  301663  461591  450004 1223896  622510  745713  628597  552875
  603306  760174  588943  589270  571312  665030  236597  314214  635139
  645897  556369  614549  608498  628567  452617  617972  603349  569463
  615702  573325  557739  257528  188898  593090  534472  617249  646627
  639216  448512  631307  541633  673113  138657   69213  418885  574632
  593050  450780  610891  277149  287741  568658  618319  573956  475955
  553239    9479  589078    7872  605676    7958  425258  172847  645853
  608833  458736  632123  560822  657093  120354  578816   20995  679949
  441927  634165  277177  279545  596043  773264  585788  577311  445654
  616372  603457  603112  605672  601985  480596  595521  619126  582004
  630058  524750  535707  465286  608504  589272  610442  618651  623771
  594304  467328  592233  709391  677285  588593  588916   69439  412371
  610912  631316  557092  569850  610736  636988  481004  515330  628560
  564853  563564  566586   91466  462373]
(122,)


In [3]:
bitrate_data = []
prev_x = []
num_time_periods = 1
bitrate = 0

num_prev_chunks = 5
period = 5

for i in range(len(df)):
    if(not("eth:ethertype:ip:udp:data" == df['frame.protocols'][i] or "eth:ethertype:ip:udp:quic" == df['frame.protocols'][i])):
        continue
    else:
        if(df['frame.time_relative'][i]>=num_time_periods*period): #more than 0.3s since last packet, means new chunk
            bitrate = bitrate/5 #get rate from total data in that 5s
            if(len(prev_x) == num_prev_chunks):
                bitrate_data.append(prev_x[:])
                prev_x.pop(0)
                prev_x.append(bitrate)
            else:
                prev_x.append(bitrate)
            bitrate = 0
            num_time_periods += 1
        else:
            bitrate += int(df['frame.len'][i])
            

bitrate_data=np.asarray(bitrate_data)
print(bitrate_data)
print(bitrate_data.shape)

[[9.913564e+05 2.905648e+05 5.497352e+05 7.261338e+05 8.738488e+05]
 [2.905648e+05 5.497352e+05 7.261338e+05 8.738488e+05 4.382618e+05]
 [5.497352e+05 7.261338e+05 8.738488e+05 4.382618e+05 1.294296e+05]
 [7.261338e+05 8.738488e+05 4.382618e+05 1.294296e+05 0.000000e+00]
 [8.738488e+05 4.382618e+05 1.294296e+05 0.000000e+00 2.175360e+05]
 [4.382618e+05 1.294296e+05 0.000000e+00 2.175360e+05 1.262230e+05]
 [1.294296e+05 0.000000e+00 2.175360e+05 1.262230e+05 1.083910e+05]
 [0.000000e+00 2.175360e+05 1.262230e+05 1.083910e+05 1.347272e+05]
 [2.175360e+05 1.262230e+05 1.083910e+05 1.347272e+05 4.163820e+04]
 [1.262230e+05 1.083910e+05 1.347272e+05 4.163820e+04 1.984610e+05]
 [1.083910e+05 1.347272e+05 4.163820e+04 1.984610e+05 2.087660e+05]
 [1.347272e+05 4.163820e+04 1.984610e+05 2.087660e+05 1.221782e+05]
 [4.163820e+04 1.984610e+05 2.087660e+05 1.221782e+05 6.480000e+01]
 [1.984610e+05 2.087660e+05 1.221782e+05 6.480000e+01 2.266858e+05]
 [2.087660e+05 1.221782e+05 6.480000e+01 2.26685